## Imports

In [9]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing import image_dataset_from_directory
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import os
from PIL import Image
from tensorflow.keras.preprocessing.image import ImageDataGenerator

## Enabling GPU

In [10]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    tf.config.experimental.set_virtual_device_configuration(gpus[0], [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=10240)])

## Getting database

In [11]:
model_path = "All-Animals.keras"
model_exists = os.path.exists(model_path)

retrain = False
train = "All-Animals/Training Data"
valid = "All-Animals/Validation Data"
test = "All-Animals/Testing Data"

## Recreating Dataset

In [12]:
if recreate:
    for path in [train, valid, test]:
        for root, dirs, files in os.walk(path):
            for index, file in enumerate(files):
                if file.lower().endswith(('.jpg', '.jpeg')):
                    old_path = os.path.join(root, file)
                    img = Image.open(old_path)
                    new_filename = f"{os.path.basename(root)}_{index + 1}.jpeg"
                    new_path = os.path.join(root, new_filename)
                    img.save(new_path, 'JPEG')
                    os.remove(old_path)

augmented_root = os.path.join(os.path.dirname(train), "Augmented Photos")

## Augmenting pictures

In [13]:
if recreate:
    for root, dirs, files in os.walk(train):
        parent_folder = os.path.basename(root)
        augmented_folder = os.path.join(augmented_root, parent_folder)
        os.makedirs(augmented_folder, exist_ok=True)

        for file in files:
            path = os.path.join(root, file.lower())
            if '_aug_' not in path:
                img = np.array(Image.open(path))
                datagen = ImageDataGenerator(rotation_range=30, width_shift_range=0.2, height_shift_range=0.2, shear_range=0.2, zoom_range=0.3, horizontal_flip=True, brightness_range=[0.8, 1.2])
                augmented_images = [next(datagen.flow(np.expand_dims(img, 0), batch_size=1))[0] for _ in range(100)]
                to = 4 if path.endswith(".jpg") else 5 if path.endswith(".jpeg") else 1
                for i, aug_img in enumerate(augmented_images):
                    aug_img_path = os.path.join(augmented_folder, f"{file[:-to]}_aug_{i+1}.jpeg")
                    Image.fromarray(aug_img.astype("uint8")).save(aug_img_path)

train = augmented_root

## Dataset Loading

In [ ]:
img_size = (299, 299)
batch_size = 32

def preprocess(image, label):
    return tf.image.random_contrast(tf.image.random_brightness(image, 0.2), 0.8, 1.2), label

train_ds = image_dataset_from_directory(train, image_size=img_size, batch_size=batch_size).map(preprocess)
val_ds = image_dataset_from_directory(valid, image_size=img_size, batch_size=batch_size)
test_ds = image_dataset_from_directory(test, image_size=img_size, batch_size=batch_size)

class_names = train_ds.class_names

## Model Definition

In [8]:
with tf.device('/GPU:0'):
    if not retrain:
        model = keras.Sequential([
            layers.Rescaling(1./255, input_shape=(256, 256, 3)),
            layers.Conv2D(32, (3, 3), activation='relu'),
            layers.MaxPooling2D(),
            layers.Conv2D(64, (3, 3), activation='relu'),
            layers.MaxPooling2D(),
            layers.Conv2D(128, (3, 3), activation='relu'),
            layers.MaxPooling2D(),
            layers.Flatten(),
            layers.Dense(128, activation='relu'),
            layers.Dense(len(class_names), activation='softmax')
        ])
        model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    epochs = 20
    history = model.fit(train_ds, validation_data=val_ds, epochs=epochs)
    model.save(model_path)

16705208/16705208 [==============================] - 2s 0us/step
Epoch 1/20
17885/17885 [==============================] - 1569s 87ms/step - loss: 2.7101 - accuracy: 0.0685 - val_loss: 2.7046 - val_accuracy: 0.0672 - lr: 0.0010
Epoch 2/20
17885/17885 [==============================] - 1558s 87ms/step - loss: 2.7060 - accuracy: 0.0687 - val_loss: 2.7046 - val_accuracy: 0.0672 - lr: 0.0010
Epoch 3/20
17885/17885 [==============================] - 1560s 87ms/step - loss: 2.7060 - accuracy: 0.0687 - val_loss: 2.7048 - val_accuracy: 0.0672 - lr: 0.0010
Epoch 4/20
17885/17885 [==============================] - 1580s 88ms/step - loss: 2.7060 - accuracy: 0.0686 - val_loss: 2.7048 - val_accuracy: 0.0672 - lr: 0.0010
Epoch 5/20
17885/17885 [==============================] - 1574s 88ms/step - loss: 2.7059 - accuracy: 0.0683 - val_loss: 2.7051 - val_accuracy: 0.0672 - lr: 5.0000e-04
Epoch 6/20
17885/17885 [==============================] - 1575s 88ms/step - loss: 2.7059 - accuracy: 0.0686 - val_lo

TypeError: Unable to serialize [2.0896919 2.1128857 2.1081853] to JSON. Unrecognized type <class 'tensorflow.python.framework.ops.EagerTensor'>.

## Test

In [ ]:
loss, accuracy = model.evaluate(test_ds)
print(f'Test Accuracy: {accuracy:.4f}')


predictions = model.predict(test_ds)
pred_labels = np.argmax(predictions, axis=1)

true_labels = np.concatenate([y.numpy() for _, y in test_ds], axis=0)

misclassified_indices = np.where(pred_labels != true_labels)[0]

num_images = min(10, len(misclassified_indices))
fig, axes = plt.subplots(1, num_images, figsize=(15, 5))

for i, idx in enumerate(misclassified_indices[:num_images]):
    img, _ = list(test_ds)[idx]
    axes[i].imshow(img.numpy().astype("uint8"))
    axes[i].axis("off")
    axes[i].set_title(f"True: {true_labels[idx]}, Pred: {pred_labels[idx]}")

plt.show()